In [ ]:
# Install necessary packages (uncomment if needed)
!pip install ultralytics
!pip install supervision

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import numpy as np
import csv
from ultralytics import YOLO
import supervision as sv
from google.colab.patches import cv2_imshow
from datetime import datetime
from shapely.geometry import Polygon, box as shapely_box

In [ ]:
# Load the model
model = YOLO('/content/drive/MyDrive/Model/training_final_upgrade2/weights/best.pt')


In [ ]:
video_path = '/content/drive/MyDrive/IMG_9295.MOV'

# Capture the video
cap = cv2.VideoCapture(video_path)
# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))


print(f"Video dimensions: {width}x{height}")
print(f"FPS: {fps}")
print(f"Total frames: {total_frames}")


In [ ]:
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 files
output_path = '/content/output_09.mp4' # Path where the output video will be saved
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

#Empty space inside ROI

In [ ]:
# Adjust detection threshold
model.conf = 0.15

# Define the callback function for inference
def callback(image: np.ndarray) -> sv.Detections:
    result = model(image)[0]  # Get the first result
    return sv.Detections.from_ultralytics(result)  # Convert to Detections object

# Define the slicer object with the callback function
slicer = sv.InferenceSlicer(callback=callback)

# Define your ROI for counting vehicles
right_roi = np.array([[400, 700], [-250, 1850], [1600, 1850], [720, 700]], np.int32)
right_roi_polygon = Polygon(right_roi.reshape(-1, 2))

# Function to calculate IoU
def box_iou(box1, box2):
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])

    inter_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)

    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area > 0 else 0


# حساب المساحة الإجمالية لمنطقة الاهتمام (ROI)
roi_area = right_roi_polygon.area

# Initialize video capture (assuming `cap` is defined)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video FPS
fps = cap.get(cv2.CAP_PROP_FPS)
frame_interval = 1

frame_count = 0
heavy_frame_counter = 0
medium_or_flow_counter = 0

# CSV file setup
csv_file = open('traffic_data.csv', mode='w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Exit', 'Vehicles Count', 'Traffic State', 'Gate State', 'Empty Area', 'Empty Percentage', 'Timestamp'])

# Gate state variable
gate_state = "Open"

# Define the classes you want to count (e.g., motorcycle and car)
vehicles_classes = ['motorcycle', 'car',"bus","truck"]

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_count % frame_interval == 0:
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Perform YOLOv8 inference on the frame
        results = model(frame)
        cv2.polylines(frame, [right_roi], True, (255, 255, 255), 2)

        vehicles_in_roi_count = 0
        current_boxes = []
        occupied_area = 0

        for result in results:
            for bbox in result.boxes:
                # Convert tensors to integers
                xyxy = bbox.xyxy.cpu().numpy().flatten()  # Get box coordinates as a flat array
                x1, y1, x2, y2 = map(int, xyxy)  # Extract coordinates
                confidence = bbox.conf.item()  # Get confidence level
                class_id = int(bbox.cls.item())  # Get class ID
                class_name = model.names[class_id]  # Get class name

                # Check if the detected class is a vehicle
                if class_name not in vehicles_classes:
                    continue

                # Bounding box as a shapely box object
                box_polygon = shapely_box(x1, y1, x2, y2)

                # Calculate the intersection between the box and the ROI
                intersection_area = right_roi_polygon.intersection(box_polygon).area

                if intersection_area > 0:
                    # Calculate percentage of bounding box inside the ROI
                    percentage_inside = (intersection_area / box_polygon.area) * 100

                    # Only count the vehicle if more than 50% of its area is inside the ROI
                    if percentage_inside > 50:
                        vehicles_in_roi_count += 1
                        current_boxes.append((x1, y1, x2, y2))
                        occupied_area += intersection_area

                        # Draw bounding box and label
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 2)
                        label = f'{class_name} {confidence:.2f}'
                        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Filter out overlapping boxes
        filtered_boxes = []
        for box in current_boxes:
            if not any(box_iou(box, other_box) > 0.5 for other_box in filtered_boxes):
                filtered_boxes.append(box)

        vehicles_in_roi_count = len(filtered_boxes)

        # حساب المساحة الفارغة
        empty_area = roi_area - occupied_area
        empty_percentage = (empty_area / roi_area) * 100

        # Determine traffic state based on the empty area percentage
        if empty_percentage < 20:
            traffic_state = 'Heavy'
            heavy_frame_counter += 1
            medium_or_flow_counter = 0  # Reset the counter for medium/flow
        elif empty_percentage < 50:
            traffic_state = 'Medium'
            heavy_frame_counter = 0  # Reset heavy frame counter
            medium_or_flow_counter += 1  # Increment the medium/flow counter
        else:
            traffic_state = 'Flow'
            heavy_frame_counter = 0  # Reset heavy frame counter
            medium_or_flow_counter += 1  # Increment the medium/flow counter

        # Gate state logic
        if heavy_frame_counter >= 2:
            gate_state = 'Closed'  # The gate closes if heavy for 60 frames
        elif medium_or_flow_counter >= 2:
            gate_state = 'Open'  # The gate opens if medium or flow for 60 frames

        # Write to CSV
        csv_writer.writerow(['Exit 7', vehicles_in_roi_count, traffic_state, gate_state, empty_area, empty_percentage, timestamp])

        # Display results on frame
        height, width, _ = frame.shape
        cv2.putText(frame, f"Vehicles in ROI: {vehicles_in_roi_count}", (10, height - 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.putText(frame, f"Gate State: {gate_state}", (10, height - 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.putText(frame, f"Time: {timestamp}", (10, height - 140), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.putText(frame, f"Traffic State: {traffic_state}", (10, height - 160), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.putText(frame, f"Empty Area: {empty_area:.2f}", (10, height - 180), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.putText(frame, f"Empty Percentage: {empty_percentage:.2f}%", (10, height - 200), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Display frame
        # cv2_imshow(frame)
        out.write(frame)

    frame_count += 1

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
csv_file.close()
